In [23]:
""" 
Fine tune a image captioning or description model on a artwork description data to
obtain a model that is able to analyze an artwork and generate a description of 
the artwork that is hopefully more art-review-like than a generic image captioning 
model would

The pretrained model used here is the Salesforce Blip Captioniong model: 
https://huggingface.co/Salesforce/blip-image-captioning-base

The fine tuning data set is the data from the SemArt Project: 
https://github.com/noagarcia/SemArt
"""

' \nFine tune a image captioning or description model on a artwork description data to\nobtain a model that is able to analyze an artwork and generate a description of \nthe artwork that is hopefully more art-review-like than a generic image captioning \nmodel would\n\nThe pretrained model used here is the Salesforce Blip Captioniong model: \nhttps://huggingface.co/Salesforce/blip-image-captioning-base\n\nThe fine tuning data set is the data from the SemArt Project: \nhttps://github.com/noagarcia/SemArt\n'

In [24]:
import os
import torch
import pandas as pd
import torch.nn as nn
import chardet
from PIL import Image
from torchvision import transforms
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import Dataset as HFDataset, DatasetDict, IterableDataset
from transformers import BlipProcessor, BlipForConditionalGeneration, ViTImageProcessor, VisionEncoderDecoderModel, AutoTokenizer, TrainingArguments, Trainer

In [25]:
def list_files_in_directory(dir_path):
    try:
        files = os.listdir(dir_path)
        return [x for x in files if '.jpg' in x]
    except FileNotFoundError:
        return f"Directory not found: {dir_path}"
    except NotADirectoryError:
        return f"Not a directory: {dir_path}"

In [26]:
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import os

# Step 1: Load the CSV file as a dictionary mapping image names to descriptions
semart_dir = f"/Users/rckyi/Documents/Datasette/SemArtData/SemArt" 
images_dir = semart_dir 

description_file_train = semart_dir + '/semart_train.csv'
description_file_test = semart_dir + '/semart_test.csv'

with open(description_file_train, 'rb') as file:
    print(f'file path {description_file_train}')
    result = chardet.detect(file.read())
    encoding = result['encoding']
    print(f'encoding {encoding}')
    df_train = pd.read_csv(description_file_train, encoding=encoding, sep='\t')
    print(type(df_train['IMAGE_FILE'][0]))
    df_test = pd.read_csv(description_file_test, encoding=encoding, sep='\t')


image_to_description_train = dict(zip(df_train["IMAGE_FILE"], df_train["DESCRIPTION"]))
image_to_description_test = dict(zip(df_test["IMAGE_FILE"], df_test["DESCRIPTION"]))
# list(image_to_description.items())[0:5]

# # Define the image directory and transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed shape
    transforms.ToTensor(),  # Convert image to tensor
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])


file path /Users/rckyi/Documents/Datasette/SemArtData/SemArt/semart_train.csv
encoding utf-8
<class 'str'>


In [27]:
image_files_in_dir = list_files_in_directory(semart_dir+'/Images/')

In [28]:
from datasets import IterableDataset, DatasetDict
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer            

def data_generator_train():
    image_file_list = list(df_train['IMAGE_FILE'])
    for image_name in image_files_in_dir:
        image_path = f"{semart_dir}/Images/{image_name}"

        if image_name in image_file_list: #image_to_description: # and os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
            image_tensor = transform(image)
            yield {"image":image_tensor, "text":image_to_description_train[image_name]}
            
def data_generator_test():
    image_file_list = list(df_test['IMAGE_FILE'])
    for image_name in image_files_in_dir:
        image_path = f"{semart_dir}/Images/{image_name}"

        if image_name in image_file_list: #image_to_description: # and os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
            image_tensor = transform(image)
            yield {"image":image_tensor, "text":image_to_description_test[image_name]}

In [29]:
hf_dataset = DatasetDict({"train": datasets.Dataset.from_generator(data_generator_train)
                         , "test": datasets.Dataset.from_generator(data_generator_test)})

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [30]:
def transforms(example_batch):
    images = np.asarray([x for x in example_batch["image"]])
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, 
                       padding="max_length",   # ✅ Pads or truncates to max_length
                        truncation=True,        # ✅ Prevents sequences longer than max_length
                        max_length=512  )
    inputs.update({"labels": inputs["input_ids"]})
    return inputs

hf_dataset.set_transform(transforms)

In [31]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./blip-semart-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    save_steps=500,
#     evaluation_strategy=None,
    evaluation_strategy="steps",
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False,  # Set to True if uploading to Hugging Face Hub
    logging_steps=10,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    max_steps=300,
    no_cuda=True,
)

/Users/rckyi/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/rckyi/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [32]:
# Load Pretrained BLIP Model & Processor
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

In [33]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [34]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = hf_dataset["train"],
    eval_dataset = hf_dataset["test"],
    tokenizer=processor,
#     data_collator=collate_fn,  # Use custom collate function to prepare batches
)

<ipython-input-34-a18405505b84>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [35]:
# Start Fine-Tuning
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: shaddie77 (shaddie77-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Step,Training Loss,Validation Loss
10,9.369800,7.868719
20,7.296600,6.785567
30,6.383000,5.883533
40,5.517100,5.028250
50,4.656100,4.190298
60,3.923400,3.416447
70,2.987600,2.727108
80,2.353900,2.169645
90,2.059800,1.779664
100,1.559600,1.551912


TrainOutput(global_step=300, training_loss=2.3670225048065188, metrics={'train_runtime': 72807.6436, 'train_samples_per_second': 0.033, 'train_steps_per_second': 0.004, 'total_flos': 1.4242167653400576e+18, 'train_loss': 2.3670225048065188, 'epoch': 0.12468827930174564})

In [36]:
# Save the fine-tuned model
trainer.save_model("./blip-semart-finetuned_model")
processor.save_pretrained("./blip-semart-finetuned_model")

print("Fine-tuning complete! Model saved at './blip-semart-finetuned_model'")

Fine-tuning complete! Model saved at './blip-semart-finetuned_model'


In [37]:
from huggingface_hub import notebook_login
notebook_login()

In [38]:
model.push_to_hub("shaddie/blip-semart-finetuned_model",
                  use_auth_token=True,
                  commit_message="fine-tuning-exercises",
                  private=True)

/Users/rckyi/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shaddie/blip-semart-finetuned_model/commit/b5bfa01cab56602b9b61d3c041f4a609047cbf3a', commit_message='fine-tuning-exercises', commit_description='', oid='b5bfa01cab56602b9b61d3c041f4a609047cbf3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shaddie/blip-semart-finetuned_model', endpoint='https://huggingface.co', repo_type='model', repo_id='shaddie/blip-semart-finetuned_model'), pr_revision=None, pr_num=None)

In [39]:
image_files_in_dir

['39799-21gritti.jpg',
 '10637-peasants.jpg',
 '44435-tributem.jpg',
 '33615-04sp1629.jpg',
 '15432-pieta.jpg',
 '18358-1509grec.jpg',
 '08684-09deadph.jpg',
 '42808-2baptism.jpg',
 '36530-port_wom.jpg',
 '18298-1012grec.jpg',
 '26541-04_0bay4.jpg',
 '03862-138sacr.jpg',
 '33578-123portr.jpg',
 '42242-0310vela.jpg',
 '20975-still_li.jpg',
 '40823-youngwo1.jpg',
 '24963-cassone4.jpg',
 '17796-701goya.jpg',
 '35499-29mythol.jpg',
 '40178-tonneman.jpg',
 '05374-flower_.jpg',
 '26742-04_1sp3f.jpg',
 '39742-08pardo.jpg',
 '42336-pastoral.jpg',
 '02998-portrait.jpg',
 '14508-flowers.jpg',
 '23852-3manet11.jpg',
 '17288-selfpo15.jpg',
 '20938-mirror.jpg',
 '39427-1antico2.jpg',
 '23573-peelin.jpg',
 '07773-50dream.jpg',
 '39151-15zianig.jpg',
 '31178-9exal01.jpg',
 '08990-05ursula.jpg',
 '19280-port_wom.jpg',
 '24212-madonna2.jpg',
 '32977-09chigi2.jpg',
 '25205-resurrec.jpg',
 '23861-3manet20.jpg',
 '03530-olindo.jpg',
 '07603-60alof.jpg',
 '31198-93restau1.jpg',
 '14964-tahiti10.jpg',
 '116

In [ ]:
from PIL import Image
import requests

im_path = semart_dir+'/Images/'+ '07930-venus_sc.jpg'
image = Image.open(im_path)
image

In [355]:
from accelerate.test_utils.testing import get_backend

device, _, _ = get_backend()
inputs = processor(images=image, return_tensors="pt").to(torch.float).to(device)
pixel_values = inputs.pixel_values.to(torch.float)
model = model.to(device)
pixel_values

tensor([[[[-1.1061, -1.0769, -1.0623,  ..., -1.1353, -1.1499, -1.1645],
          [-1.2813, -1.2083, -1.1499,  ..., -1.1645, -1.1645, -1.1645],
          [-1.3981, -1.3981, -1.3543,  ..., -1.1499, -1.1207, -1.1207],
          ...,
          [ 0.1639,  0.2661,  0.2369,  ..., -1.4127, -1.2813, -1.1061],
          [ 0.0763, -0.0842, -0.1718,  ..., -1.3689, -1.2375, -1.0477],
          [-0.7412, -0.8580, -0.9602,  ..., -1.3543, -1.2667, -1.0477]],

         [[-0.9867, -0.9867, -0.9867,  ..., -1.0167, -1.0317, -1.0467],
          [-1.1518, -1.0918, -1.0617,  ..., -1.0767, -1.0767, -1.0918],
          [-1.2418, -1.2418, -1.2268,  ..., -1.1068, -1.0767, -1.0767],
          ...,
          [-0.2063, -0.1313, -0.1763,  ..., -1.2418, -1.1668, -1.0317],
          [-0.2363, -0.4164, -0.4764,  ..., -1.1818, -1.1068, -0.9717],
          [-0.9117, -0.9717, -0.9717,  ..., -1.1368, -1.0767, -0.9567]],

         [[-0.8119, -0.8261, -0.8261,  ..., -0.7692, -0.7834, -0.7977],
          [-0.9541, -0.8972, -

In [356]:
# model.generate(**inputs, num_beams=4, do_sample=True)

generated_ids = model.generate(pixel_values=pixel_values, max_length=150, num_beams=1, do_sample=True)
generated_ids

tensor([[30522,  2011, 13471,  2532,  1005,  1055,  4169,  1010,  4760,  2010,
          8085,  1010,  1037,  2358,  8516,  4590,  2007,  1037,  5957,  2369,
          2032,  1010,  2003,  1996,  3395,  1998,  2069,  1997,  2029,  2027,
          2467,  2272,  2000,  2022,  1996,  2556,  2013,  3304,  1012,  1999,
          2010,  3395,  2001,  2141,  2005,  1996,  2117,  2095,  1997,  2010,
          6480,  1999,  1996,  4440,  3723,  2818,  1005,  1055,  6533,  2012,
          1996,  3653, 12662,  2050,  1010,  2002,  2003,  1037,  2266,  1997,
          1996,  3063,  1012,  1999,  2023,  4169,  1010,  2030,  2074,  1037,
          2103,  1997,  2624,  9004, 18981, 12898, 26247,  1010,  2006,  1996,
          2237,  1010,  1996,  2521,  3023,  1997, 10553,  1012,  2002,  2038,
          2042,  2170,  2000,  2022,  2019,  6422,  1012,  2011,  2010,  3008,
          1997,  1996,  2358,  8516, 27390,  2050,  1010,  2002,  2038,  2042,
          4217,  2000,  6773,  1996,  2157,  1997,  

In [357]:
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_caption

["byzanna ' s painting, showing his signature, a stylger with a landscape behind him, is the subject and only of which they always come to be the present from italy. in his subject was born for the second year of his lifetime in the triptych ' s portrait at the predella, he is a member of the artist. in this painting, or just a city of san petropolomeo, on the town, the far coast of naples. he has been called to be an elder. by his parents of the stylgura, he has been chosen to paint the right of the city in venice from the city of naples. the work has been cut down by the most significant and has been taken"]

In [358]:
model.generation_config

GenerationConfig {
  "bos_token_id": 30522,
  "eos_token_id": 2,
  "pad_token_id": 0
}

In [359]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the model and tokenizer
# model_name = "Vamsi/T5_Paraphrase_Paws"
model_refine_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_refine_name)
model_refine = AutoModelForSeq2SeqLM.from_pretrained(model_refine_name)



In [360]:
# Input sentence
sentence = generated_caption

# Prepare input with a paraphrasing prompt
input_text = f"paraphrase: {sentence} </s>"
input_ids = tokenizer.encode(input_text, return_tensors="pt")


In [361]:
# Generate refined sentence
outputs = model_refine.generate(input_ids, max_length=128, num_beams=10, early_stopping=False)
outputs[0]

tensor([    0,   784,   121,   969,   172, 10878,     3,    31,     3,     7,
         3924,     6,  2924,   112,  5483,     6,     3,     9, 22437,  1304,
           28,     3,     9,  3283,  1187,   376,     6,    19,     8,  1426,
           11,   163,    13,    84,    79,   373,   369,    12,    36,     8,
          915,    45,    34,     9,   120,     5,    16,   112,  1426,    47,
         2170,    21,     8,   511,   215,    13,   112,  6556,    16,     8,
         1469,    17, 14731,     3,    31,     3,     7,  7956,    44,     8,
          554,    26,  5303,     6,     3,    88,    19,     3,     9,  1144,
           13,     8,  2377,     5,    16,    48,  3924,     6,    42,   131,
            3,     9,   690,    13,     3,     7,   152,   158, 12395,    32,
           40,  7159,    32,     6,    30,     8,  1511,     6,     8,   623,
         4939,    13, 12522,   965,     1])

In [362]:
refined_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

refined_sentence

'["byzanna \' s painting, showing his signature, a stylger with a landscape behind him, is the subject and only of which they always come to be the present from italy. in his subject was born for the second year of his lifetime in the triptych \' s portrait at the predella, he is a member of the artist. in this painting, or just a city of san petropolomeo, on the town, the far coast of naples'